In [142]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
from pyspark.ml.classification import LogisticRegression

In [150]:
#preprocess the dataset
def preProcessData(dataset):
    
    # only keeping successful and failed projects
    projects = dataset[(dataset['state'] == 'failed') | (dataset['state'] == 'successful')]
    
    # deleting unnecesary columns
    projects.drop('ID',axis=1,inplace=True)
    projects.drop('name',axis=1,inplace=True)
    projects.drop('category',axis=1,inplace=True)
    projects.drop('currency',axis=1,inplace=True)
    projects.drop('goal',axis=1,inplace=True)
    projects.drop('pledged',axis=1,inplace=True)
    projects.drop('usd pledged',axis=1,inplace=True)
    
    # Finding length of the project 
    projects['launched'] = pd.to_datetime(projects['launched']).dt.to_period("M")
    projects['deadline'] = pd.to_datetime(projects['deadline']).dt.to_period("M")
    
    # Creating a new columns with Campaign total months
    projects['total_months'] = projects['deadline']- projects['launched']
    projects['total_months'] = projects['total_months'].astype(int)
    projects.drop('launched',axis=1,inplace=True)
    projects.drop('deadline',axis=1,inplace=True)

    # Moving the outcome variable to last and encoding it
    state = projects.pop('state')
    projects['state']=state
    projects['state']= LabelEncoder().fit_transform(projects['state'])
    
    # One Hot Encoding the categorical features
    onehotencoder = ce.OneHotEncoder(cols = ['main_category','country'])
    projects = onehotencoder.fit_transform(projects)
    
    # Avoiding dummy variable trap
    projects.drop('main_category_-1',axis=1,inplace=True)
    projects.drop('country_-1',axis=1,inplace=True)
  
    labels = projects.columns

    X = projects.iloc[:1000, :-1].values
    y = projects.iloc[:1000, -1].values

    # Splitting the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    
    test_dataf = pd.concat([pd.DataFrame(X_test),pd.Series(y_test)],axis=1)
    test_dataf.columns= labels
    
    # Feature Scaling
    sc_X = StandardScaler()
    X_train[:,38:] = sc_X.fit_transform(X_train[:,38:])
    X_test[:,38:] = sc_X.transform(X_test[:,38:])

    training_data = pd.concat([pd.DataFrame(X_train),pd.Series(y_train)],axis=1)
    test_data = pd.concat([pd.DataFrame(X_test),pd.Series(y_test)],axis=1)

    training_data.columns = labels
    test_data.columns= labels
    
    return training_data,test_data,test_dataf

In [158]:
def preProcessData1(dataset):
    
    # only keeping successful and failed projects
    projects = dataset[(dataset['state'] == 'failed') | (dataset['state'] == 'successful')]
    
    # deleting unnecesary columns
    projects.drop('ID',axis=1,inplace=True)
    projects.drop('name',axis=1,inplace=True)
    projects.drop('category',axis=1,inplace=True)
    projects.drop('currency',axis=1,inplace=True)
    projects.drop('goal',axis=1,inplace=True)
    projects.drop('pledged',axis=1,inplace=True)
    projects.drop('usd pledged',axis=1,inplace=True)
    
    # Finding length of the project 
    projects['launched'] = pd.to_datetime(projects['launched']).dt.to_period("M")
    projects['deadline'] = pd.to_datetime(projects['deadline']).dt.to_period("M")
    
    # Creating a new columns with Campaign total months
    projects['total_months'] = projects['deadline']- projects['launched']
    projects['total_months'] = projects['total_months'].astype(int)
    projects.drop('launched',axis=1,inplace=True)
    projects.drop('deadline',axis=1,inplace=True)
    
    # Moving the outcome variable to last and encoding it
    state = projects.pop('state')
    projects['state']=state
    projects['state']= LabelEncoder().fit_transform(projects['state'])
    
    Train = projects[:1000]
    Test = projects [1000:1200]

    # One Hot Encoding the categorical features
    onehotencoder = ce.OneHotEncoder(cols = ['main_category','country'])
    projects = onehotencoder.fit_transform(projects)
    
    # Avoiding dummy variable trap
    projects.drop('main_category_-1',axis=1,inplace=True)
    projects.drop('country_-1',axis=1,inplace=True)

  
    labels = projects.columns

    X_train = projects.iloc[:1000, :-1].values
    y_train = projects.iloc[:1000, -1].values
    X_test = projects.iloc[1000:1200, :-1].values
    y_test =  projects.iloc[1000:1200, -1].values

    #test_dataf = pd.concat([pd.DataFrame(X_test),pd.Series(y_test)],axis=1)
    # Feature Scaling
    sc_X = StandardScaler()
    X_train[:,38:] = sc_X.fit_transform(X_train[:,38:])
    X_test[:,38:] = sc_X.transform(X_test[:,38:])

    training_data = pd.concat([pd.DataFrame(X_train),pd.Series(y_train)],axis=1)
    test_data = pd.concat([pd.DataFrame(X_test),pd.Series(y_test)],axis=1)

    training_data.columns = labels
    test_data.columns= labels
    
    return training_data,test_data,Test

In [171]:
print(test_dataf)

     main_category  backers country  usd_pledged_real  usd_goal_real  \
1136         Music       51      US           3500.00        3500.00   
1137          Food       15      US           1190.00       15000.00   
1138        Design       91      US           4809.00       19000.00   
1140           Art       21      US           3100.00        3000.00   
1141    Publishing        3      US             35.00        5500.00   
1142    Publishing       33      US          11974.00       50000.00   
1143         Music        1      US             10.00        1250.00   
1144       Fashion        1      GB             15.05     1505185.36   
1145          Food        0      US              0.00        1000.00   
1146    Journalism        0      US              0.00        8000.00   
1147  Film & Video        2      US             30.00        4000.00   
1148         Music       16      CA           1228.92        1193.13   
1149         Games        0      US              0.00        100

In [157]:
#dataset.pop('main_category')
# One Hot Encoding the categorical features
#onehotencoder = ce.OneHotEncoder(cols = ['main_category'])
#onehotencoder.fit(dataset)
#onehotlabels = onehotencoder.transform(dataset).values.tolist()
#print(onehotlabels)

In [170]:
#Load training and test data 
dataset = pd.read_csv('ks-projects-201801.csv')
training_data, test_data,test_dataf = preProcessData1(dataset)

C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 

In [156]:
#Load training and test data 
dataset = pd.read_csv('ks-projects-201801.csv')
training_data, test_data,test_dataf = preProcessData1(dataset)

C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Tejaswini\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 

ValueError: Found array with 0 feature(s) (shape=(1000, 0)) while a minimum of 1 is required by StandardScaler.

In [160]:
print(test_dataf)

     main_category  backers country  usd_pledged_real  usd_goal_real  \
1136         Music       51      US           3500.00        3500.00   
1137          Food       15      US           1190.00       15000.00   
1138        Design       91      US           4809.00       19000.00   
1140           Art       21      US           3100.00        3000.00   
1141    Publishing        3      US             35.00        5500.00   
1142    Publishing       33      US          11974.00       50000.00   
1143         Music        1      US             10.00        1250.00   
1144       Fashion        1      GB             15.05     1505185.36   
1145          Food        0      US              0.00        1000.00   
1146    Journalism        0      US              0.00        8000.00   
1147  Film & Video        2      US             30.00        4000.00   
1148         Music       16      CA           1228.92        1193.13   
1149         Games        0      US              0.00        100

In [62]:
#compute sigmoid value
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [63]:
#compute y value for given x and weight(beta) parameter
def compute_h(x,W):
    x = np.append([1],[x])
    #perform dot product of x and W
    z=np.dot(x,W)
    return sigmoid(z)

In [64]:
#compute gradient value
def compute_gradient(point):
    #create feature vector x
    x_vector = np.append([1],[point[1:]])
    #create weight vector w
    w_vector= point[0]
    #perform dot product of w and x 
    gradient = w_vector * x_vector
    return gradient

In [65]:
#compute accuracy
def accuracy():
    # compute difference between predicted y and actual y value
    predTest_Rdd = pred_Rdd.zip(test_yRdd) 
    #absRdd = predTest_Rdd.map(lambda x : ('key',(np.abs(x[0] - x[1]))))
    absRdd = predTest_Rdd.map(lambda x : ('key',show(x)))
    meanRdd = absRdd.reduceByKey(lambda x,y :  np.add(x,y))
    accuracy= meanRdd.lookup('key')[0]/testdata_length
    return accuracy[0]

In [66]:
def show(point):
    val = np.abs(x[0] - x[1]) 
    print(val)
    return valfor x in predTest_Rdd:
    val = np.abs(x[0] - x[1]) 
    print(val)

SyntaxError: invalid syntax (<ipython-input-66-aa1f157c8448>, line 4)

In [67]:
#Load train data into numpy array
train_X = training_data.iloc[:,:-1].as_matrix()
train_y = training_data.iloc[:,-1].as_matrix()
#build weight vector with initial values as zeroes
W = np.zeros([train_X.shape[1]+1, 1])
learning_rate = 0.5

In [68]:
# convert train data loaded in nparray to rdd
train_XRdd = sc.parallelize(train_X)
train_yRdd = sc.parallelize(train_y)

In [69]:
for i in range(10):
    #compute h(x,W)
    pred_yRdd =train_XRdd.map(lambda x : compute_h(x,W))
    # combine predicted y value rdd and actual y value rdd
    predTrain_Rdd = pred_yRdd.zip(train_yRdd) 
    # compute difference between predicted y and actual y value
    devRdd = predTrain_Rdd.map(lambda x : (x[0] - x[1]))
    # compute gradient : (predicted - y)X
    devX_Rdd = devRdd.zip(train_XRdd)
    gradRdd = devX_Rdd.map(lambda lst : ('keyA',compute_gradient(lst)))
    dwRdd = gradRdd.reduceByKey(lambda x,y :  np.add(x,y))
    gradientRdd = dwRdd.mapValues(lambda dw : learning_rate * dw)
    #update weights
    weightRdd = sc.parallelize(W)
    dRdd = sc.parallelize(gradientRdd.lookup('keyA')[0])
    wRdd = weightRdd.zip(dRdd).map(lambda x : (x[0] - x[1]))
    W = np.stack(wRdd.collect())

In [70]:
#Load test data into numpy array
test_X = test_data.iloc[:20,:-1].as_matrix()
testdata_length = test_X.shape[0]
test_y = test_data.iloc[:20,-1].as_matrix()

In [71]:
#convert test data loaded in nparray to rdd
test_XRdd = sc.parallelize(test_X)
test_yRdd = sc.parallelize(test_y)

In [72]:
#predicting y value for given input test data using weight parameters learned by model
pred_Rdd =test_XRdd.map(lambda x : compute_h(x,W))

In [99]:
# compute difference between predicted y and actual y value
predTest_Rdd = pred_Rdd.zip(test_yRdd) 
#x=predTest_Rdd.zipWithIndex().collect()
#absRdd = predTest_Rdd.map(lambda x : ('key',(np.abs(x[0] - x[1]))))
#absRdd = predTest_Rdd.map(lambda x : ('key',show(x)))
#meanRdd = absRdd.reduceByKey(lambda x,y :  np.add(x,y))
#accuracy= meanRdd.lookup('key')[0]/testdata_length
#print(x)

In [100]:
 #print(x[5][-1])

In [161]:
c=[]
ic=[]
i=0
for x in predTest_Rdd.collect():
    val = np.abs(x[0] - x[1])
    #print(x[0],x[1])
    if(val == 0):
        c.append(i)
    else:
        ic.append(i)
    i+=1
print(c,ic)

[0, 1, 6, 7, 8, 10, 11, 16, 19] [2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18]


In [172]:
for i in c:
    print(test_dataf.iloc[[i]])
test_dataf.head()

     main_category  backers country  usd_pledged_real  usd_goal_real  \
1136         Music       51      US            3500.0         3500.0   

      total_months  state  
1136             1      1  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1137          Food       15      US            1190.0        15000.0   

      total_months  state  
1137             1      0  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1143         Music        1      US              10.0         1250.0   

      total_months  state  
1143             2      0  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1144       Fashion        1      GB             15.05     1505185.36   

      total_months  state  
1144             1      0  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1145          Food        0      US               0.0         1000.0   

      total_months  state  
1145             1     

,main_category,backers,country,usd_pledged_real,usd_goal_real,total_months,state
1136,Music,51,US,3500.0,3500.0,1,1
1137,Food,15,US,1190.0,15000.0,1,0
1138,Design,91,US,4809.0,19000.0,1,0
1140,Art,21,US,3100.0,3000.0,1,1
1141,Publishing,3,US,35.0,5500.0,2,0


In [163]:
for i in ic:
    print(test_dataf.iloc[[i]])


     main_category  backers country  usd_pledged_real  usd_goal_real  \
1138        Design       91      US            4809.0        19000.0   

      total_months  state  
1138             1      0  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1140           Art       21      US            3100.0         3000.0   

      total_months  state  
1140             1      1  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1141    Publishing        3      US              35.0         5500.0   

      total_months  state  
1141             2      0  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1142    Publishing       33      US           11974.0        50000.0   

      total_months  state  
1142             1      0  
     main_category  backers country  usd_pledged_real  usd_goal_real  \
1146    Journalism        0      US               0.0         8000.0   

      total_months  state  
1146             2     

In [103]:
#for x in test_y:
#    print(x)

In [107]:
print("Accuracy of Logistic Regression Model: ",100-accuracy(), "%")

Accuracy of Logistic Regression Model:  0.0 %
